In [3]:
import pandas as pd
import pyodbc

In [6]:
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
            "Server=MSI\SQLEXPRESS;"
            "Database=salary;"
            "Trusted_Connection=yes;")

In [7]:
cursor =  conn.cursor()

In [8]:
df = pd.read_csv('data.csv')

In [12]:
df['updated_salary'] = df['salary']*df['salary_increment']/100 + df['salary']

In [14]:
dep_df = df[['dept_name', 'salary_increment']].drop_duplicates().reset_index()
dep_df['department_id'] = dep_df['index']+1
del dep_df['index']

In [29]:
dep_dic = dict(zip(dep_df.dept_name , dep_df.department_id))
df['department_id'] = df['dept_name'].replace(dep_dic)

{'Finance': 1, 'IT': 2, 'Sales': 3, 'Marketing': 4}

In [31]:
DF

,first_name,last_name,salary,dept_name,salary_increment,updated_salary,department_id
0,Darius,Mufutau,3901,Finance,10,4291.10,1
1,Tiger,Elliott,5489,IT,15,6312.35,2
2,Malik,Macaulay,5444,Sales,17,6369.48,3
3,Ali,Vance,8993,Marketing,16,10431.88,4
4,Randall,Deacon,9515,IT,15,10942.25,2
5,Josiah,Lee,8113,Sales,17,9492.21,3
6,Dante,Mohammad,8446,Sales,17,9881.82,3
7,Wyatt,Kuame,4817,Marketing,16,5587.72,4
8,Quinn,Oliver,5513,Finance,10,6064.30,1
9,Oliver,Gary,5158,IT,15,5931.70,2


In [32]:
dep_df

,dept_name,salary_increment,department_id
0,Finance,10,1
1,IT,15,2
2,Sales,17,3
3,Marketing,16,4


In [46]:
cursor.execute('drop table updated_salaries;')
cursor.execute('drop table employee;')
cursor.execute('drop table department;')

In [47]:
cursor.execute(
    """CREATE TABLE department (
    id INT  PRIMARY KEY not null,
    name TEXT not null,
    salary_increment NUMERIC not null, 
    );
    """)

cursor.execute(
    """CREATE TABLE updated_salaries (
    employee_id INT UNIQUE not null,
    updated_salary NUMERIC not null, 
    );
    """)

cursor.execute(
    """CREATE TABLE employee (
    id INT  PRIMARY KEY not null,
    first_name TEXT not null,
    last_name TEXT not null,
    salary NUMERIC not null ,
    department_id int FOREIGN KEY REFERENCES department(ID) not null
    );
    """)
cursor.commit()

In [53]:
for i , row in df.iterrows():
        insertSql = "insert into updated_salaries (employee_id, updated_salary) values (?, ?)"
        cursor.execute(insertSql, i+1 , row.updated_salary)
        cursor.commit()
    
for i , row in df.iterrows():
        insertSql = "insert into employee (id, first_name,last_name, salary, department_id) values (?,?,?,?,?)"
        cursor.execute(insertSql, i+1 , row.first_name, row.last_name , row.salary , row.department_id)
        cursor.commit()

for i , row in dep_df.iterrows():
        insertSql = "insert into department  (id, name, salary_increment) values (?,?,?)"
        cursor.execute(insertSql, int(row.department_id) , row.dept_name , row.salary_increment)
        cursor.commit()